In [4]:
import os

from groq import Groq
from dotenv import load_dotenv

load_dotenv() 

client = Groq(
    api_key=os.getenv("GROQ_API_KEY")
)

In [57]:
def get_key_phrase(text):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f'''
                from the following text, extract the keyphrases and write them down as an array, be careful to keep the phrases intact, do not edit them. Note that keyphrases are often longer than keywords and they can help learners understand the main idea of ​​the text by reading only the keyphrases. There may be no keyphrases if the text has no specific content:
                Write only an array containing keyphrases or an empty array. Do not write any additional information. Do not introduce. Just generate [...]
                text:
                {text}''',
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    return chat_completion.choices[0].message.content

In [25]:
import re

def extract_text_between_brackets(text):
    # Sử dụng regex để tìm các đoạn văn bản nằm giữa [ và ]
    result = re.findall(r'\[(.*?)\]', text)
    if not result:
        return '<NONE>'
    return result[0]

In [16]:
import re

def remove_puctuation(text):
    return re.sub(r'[^\w\s]', '', text)

In [23]:
import ast

def get_final_keyphrase(text):
    llm_out = get_key_phrase(text)
    keyphrases = extract_text_between_brackets(llm_out).split('", "')
    keyphrases = [s.strip('"') for s in keyphrases]
    for key in keyphrases:
        key = remove_puctuation(key)
    return keyphrases

In [18]:
def label_keyphrase(texts, keyphrases):
    # Khởi tạo mảng label với giá trị mặc định là 0, độ dài bằng số từ trong câu
    
    labels = [0] * len(texts)

    # Xử lý từng keyphrase trong mảng keyphrases
    for keyphrase in keyphrases:
        keyphrase_tokens = keyphrase.split()
        n = len(keyphrase_tokens)

        # Tìm vị trí bắt đầu của keyphrase trong câu
        for i in range(len(texts) - n + 1):
            if texts[i:i + n] == keyphrase_tokens:
                # Gán giá trị 1 cho các vị trí thuộc keyphrase
                for j in range(i, i + n):
                    labels[j] = 1

    return labels

In [19]:
import json

# Open and read the JSON file
with open('data.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

In [52]:
for d in data[4500:5000]:
    text = ' '.join(d['text'])
    keyphrases = get_final_keyphrase(text)
    d['keyphrase'] = keyphrases
    labels = label_keyphrase(d['text'], keyphrases)
    d['label'] = labels

In [53]:
with open('new_data.json', 'w', encoding='utf-8') as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

In [55]:
' '.join(data[5034]['text'])

'Nếu cô không nhớ được số tiền bé đấy thì cô cho cháu đi, cháu hứa lấy một nửa số tiền đấy chuyển thằng vào tài khoản của MTTQ. Thề luôn uy tín.'

In [86]:
def gen_comments():
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": '''
                Generate exactly 50 Vietnamese specific random comments on Social Media. Write only an array containing comments. Do not write any additional information.
                Do not write " "
                ''',
            }
        ],
        model="llama-3.1-70b-versatile",
    )
    return chat_completion.choices[0].message.content


In [87]:
s = gen_comments()

In [88]:
arr = re.findall(r'\[(.*?)\]', s, re.DOTALL)

In [89]:
arr

['Trời ơi, món này ngon quá,\nMình vừa xem phim hay lắm,\nBây giờ thời tiết quá nóng,\nChả giò này bắt mắt quá,\nEm đã sẵn sàng cho kì thi,\nĐội tuyển bóng đá Việt Nam thật tuyệt,\nCơm tấm Sài Gòn ngon không cưỡng,\nMình thích đi biển quá,\nHà Nội mùa thu thật đẹp,\nCây bàng này quá to,\nBữa này mình ăn gì đây,\nMón này trông ngon quá,\nEm thích đọc sách của Nguyễn Nhật Ánh,\nĐi chợ dân sinh quá thú vị,\nRural Vietnam thật đẹp,\nMình thích nghe nhạc Trịnh Công Sơn,\nMón xôi này ngon lắm,\nThật tuyệt khi được đi du lịch,\nCâu view ở Đà Lạt thật đẹp,\nTrà sữa Trà Sữa 3A quá ngon,\nBún riêu này bắt mắt quá,\nMình thích món Phở này,\nChả giò này quá ngon,\nSang Sưa mùa hè này HOT quá,\nCaffe Sữa Đá ở Hà Nội thật ngon,\nEm thích ăn cơm Tấm Sài Gòn,\nBảo tàng lịch sử Việt Nam thật hay,\nTôi thích nhìn cảnh thật đẹp,\nEm thích chăm sóc cây hoa,\nTiệc tùng cuối năm thật vui,\nCây xương rồng thật đẹp,\nChuyến đi Đà Nẵng quá thú vị,\nLại thích thời trang đường phố Sài Gòn,\nMình thích chơi cờ,\n

In [90]:
len(arr[0].split(',\n'))

48

In [92]:
arr[0].split(',\n')[0]

'Trời ơi, món này ngon quá'

In [ ]:
def gen_data():
    llm_out = gen_comments()
    arr = re.findall(r'\[(.*?)\]', llm_out, re.DOTALL)
    comments = arr[0].split('\n')
    return comments